# Notebook imports

In [23]:
import pandas as pd
import numpy as np

# Constants

In [24]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/03_Testing/test-data.txt'
VOCAB_SIZE = 2500


# Read and Load Featres from .txt Files int np array

In [25]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE,delimiter=' ',dtype=int)

In [26]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [27]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [28]:
sparse_train_data[-5:]


array([[5795, 2040,    0,    2],
       [5795, 2041,    0,    1],
       [5795, 2277,    0,    1],
       [5795, 2325,    0,    1],
       [5795, 2381,    0,    1]])

In [29]:
print('No of rows in training file',sparse_train_data.shape[0])
print('No of rows in test file',sparse_test_data.shape[0])


No of rows in training file 258364
No of rows in test file 117786


In [30]:
print('No of emails in training file',np.unique(sparse_train_data[:,0]).size)

No of emails in training file 4013


In [31]:
print('No of emails in test file',np.unique(sparse_test_data[:,0]).size)


No of emails in test file 1724


### How to create an Empty Dataframe

In [32]:
column_names = ['DOC_ID']+['CATEGORY']+ list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [33]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [34]:
full_train_data = pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)

In [35]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create A Full Matrix from a Sparse matrix

In [36]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a panda dataframe
    Keyword arguments
    sparse matrix -- numpy array
    nr_words -- size of the vocabulary.Total no. of tokens
    doc_idx -- position of document id in the sparse matrix. Default: 1st column
    word_idx -- position of word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of label (spam is 1, nonspam is 0) in the sparse matrix. Default: 3rd column
    freq_idx -- position of occurence of word id in the sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID']+['CATEGORY']+ list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)

    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label  = sparse_matrix[i][cat_idx]
        occurence = sparse_matrix[i][freq_idx]

        full_matrix.at[doc_nr,'DOC_ID']=doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr,word_id] = occurence

    full_matrix.set_index('DOC_ID',inplace=True)



    return full_matrix

In [37]:
%%time
full_train_data =make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 12.3 s


In [38]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating probability of a spam

In [39]:
full_train_data.CATEGORY.size

4013

In [40]:
full_train_data.CATEGORY.sum()

1248

In [42]:
prrob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print('Probability of spam is: ',prrob_spam)

Probability of spam is:  0.310989284824321


### Total Number of Words/Tokens

In [44]:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [46]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [48]:
total_wc =email_lengths.sum()
total_wc

429235

## Number of tokens in spam and ham

In [49]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [50]:
spam_wc = spam_lengths.sum()
spam_wc

176339

In [51]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [52]:
email_lengths.shape[0]- spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [53]:
nonspam_wc = ham_lengths.sum()

In [54]:
spam_wc + nonspam_wc - total_wc

0

In [59]:
print('Average no of words in spam emails {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Average no of words in ham emails {:.3f}'.format(nonspam_wc/ham_lengths.shape[0]))

Average no of words in spam emails 141
Average no of words in ham emails 91.463
